In [1]:
import psfmachine as psf
import lightkurve as lk
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, Angle
from astroquery.gaia import Gaia
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.io import fits
import warnings
from scipy import sparse

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=sparse.SparseEfficiencyWarning)

%matplotlib widget

In [2]:
def query_gaia_region(ra, dec, radius_deg, maglim = 17):
    # Define the central coordinates of your region
    Gaia.ROW_LIMIT = 20
    coords = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')

    # Define the search radius (half of the square side length)
    radius = radius_deg * u.degree

    # Query Gaia data
    job = Gaia.cone_search_async(coords, radius=radius)
    result = job.get_results().to_pandas()

    result = result[result['phot_g_mean_mag'] < maglim]

    df_first_row = result.iloc[[0]].reset_index(drop=True)

    return df_first_row


In [3]:
ra = 202.9625
dec = -10.736

result = query_gaia_region(ra, dec, 3, maglim = 17)
print(result['ra'], result['dec'])
print(type(result))

target_str = str(ra) + ' ' + str(dec)

tpfs = lk.search_targetpixelfile(target_str, mission='K2', campaign = 6, radius=2, limit=6, cadence='long').download_all(quality_bitmask=None)

print(tpfs)

2024-06-24 01:20:40,260 - astroquery - INFO - Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
0    202.968879
Name: ra, dtype: float64 0   -10.759365
Name: dec, dtype: float64
<class 'pandas.core.frame.DataFrame'>
TargetPixelFileCollection of 1 objects:
    0: KeplerTargetPixelFile Object (ID: 212593538)


In [4]:
machine = psf.TPFMachine.from_TPFs(tpfs)
machine

2024-06-24 01:20:51,065 - astroquery - INFO - Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
Querying Gaia: 1 sources found.
Init sources: 1
pre-comp sources 1
comps: 1 1
line 240


TPFMachine (N sources, N times, N pixels): (1, 3544, 63)

In [5]:
machine.remove_background_model(plot=True)
plt.show()

source_rad: [7.20448699]
source_flux_estimates: 13651.763646284264
f: [[1.70180512 1.35836522 1.29929349 1.74795384 1.18446098 1.20774146
  1.4429423  1.91438758 1.62240968 1.44825285 1.49980891 1.75862342
  2.1886202  1.60090764 1.61559192 1.43224868 2.23801144 1.68764781
  1.96732556 2.30095434 2.23941836 2.02402315 2.35890039 1.49617159
  1.65480786 2.03426589 2.36464393 2.85436369 2.90848927 2.65935475
  2.10521442 1.86683562 1.87752606 2.22357017 2.46042495 2.97531438
  2.89252771 2.52068123 2.20331619 1.81021907 2.78473293 1.75996307
  2.2332958  2.07854356 2.11502483 1.92499904 1.77728484 1.60560873
  2.79135989 2.58390103 2.78175257 2.45873142 1.88635107 1.71406262
  1.48579879 2.1913802  0.71211523 1.49302246 1.56207803 2.0759129
  1.66281362 1.6749766  1.21738634]]
test_val: [[3.02136981 3.01827972 3.01518963 ... 2.74017173 2.73708164 2.73399155]
 [3.02136981 3.01827972 3.01518963 ... 2.74017173 2.73708164 2.73399155]
 [3.02136981 3.01827972 3.01518963 ... 2.74017173 2.737081

/Users/zgl12/anaconda3/lib/python3.11/site-packages/psfmachine/machine.py:507: RankWarning: Polyfit may be poorly conditioned
  np.polyfit(test_f[ok], l[ok], 1), np.log10(self.source_flux_estimates)


ZeroDivisionError: Weights sum to zero, can't be normalized